
#Credit Code Tool to [@Hanaminikata](https://www.youtube.com/@Hanaminikata)
#Ported to Colab by [Nam](https://github.com/IAmNamNguyen)
---
Cấu hình của Google Colab miễn phí:
13GB RAM, CPU Intel Xeon 2 nhân 2,2GHz, 100GB Bộ nhớ trong.

Vì CPU chỉ có 2 nhân nên mình nghĩ không nên chạy quá nhiều luồng hay tiến trình, các bạn chỉ nên chạy 2-3 Tiến trình HOẶC 5-6 Luồng.

Có thể xảy ra bug nên các bạn thông cảm (>﹏<。)

---
Specifications of Google Colab free:
13GB of RAM, 2.2GHz dual-core Intel Xeon CPU, 100GB of Internal Memory.

Because the CPU only has 2 cores, I think you should not run too many threads or processes, you should only run 2-3 Processes OR 5-6 Threads.

Bugs may occur so please understand (>﹏<。)



In [ ]:
#@title Khởi tạo môi trường chạy / Initialize the runtime environment

#https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com
%%shell
lscpu
grep MemTotal /proc/meminfo
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

# Install Needed Libraries
pip install selenium
pip install requests
pip install pillow
pip install pytesseract pillow selenium requests
pip install selenium pytesseract pillow
pip install pytesseract
pip install tesseract
apt install tesseract-ocr
apt install libtesseract-dev

touch acc.txt

In [ ]:
#@title Tool (Tiến trình / Processes)

import pytesseract
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import time
from multiprocessing import Process
import os




def hide_errors():
    if os.name == 'posix':
        os.system('echo "\033[2J"')
    elif os.name == 'nt':
        os.system('cls')

hide_errors()

class GFN_TU_DONG:
    def __init__(self, x, y, width, height):
        self.mat_khau = 'hanamidz123'
        self.mail = ''
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.driver = None
        self.success_count = 0

    @staticmethod
    def recognize_captcha(captcha_image_path):
        captcha_image = Image.open(captcha_image_path)
        captcha_text = pytesseract.image_to_string(captcha_image)
        return captcha_text

    def get_mail(self):
        url = 'https://www.1secmail.com/api/v1/?action=genRandomMailbox&count=1'
        res = requests.get(url).json()
        self.mail = res[0]
        return self.mail

    def check_mail(self, login, domain):
        url = f'https://www.1secmail.com/api/v1/?action=getMessages&login={login}&domain={domain}'
        res = requests.get(url).json()
        id = res[0]['id']
        return id

    def code_mail(self, login, domain, id):
        url2 = f"https://www.1secmail.com/api/v1/?action=readMessage&login={login}&domain={domain}&id={id}"
        getip_new = requests.get(url2).json()
        body = getip_new['body']
        code_start = body.find('驗證碼')
        code = body[code_start:].split(':')[1].strip()
        return code

    def tao_acc(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        service = Service(executable_path=r'/usr/bin/chromedriver')
        driver = webdriver.Chrome(service=service,options=options)
        driver.set_window_rect(self.x, self.y, self.width, self.height)

        driver.get('https://gfn.taiwanmobile.com/portal/index')
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/nav/div[2]/ul[1]/div[2]/a[1]/li'))).click()
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/p[3]/a'))).click()
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/p[2]/a'))).click()
        self.get_mail()
        login, domain = self.mail.split('@')
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/span/div/input'))).send_keys(self.mail)
        print(self.mail)
        print('-----------------')

        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[2]/a'))).click()

        while True:
            for i in range(10):
                try:
                    sleep(10)
                    id = self.check_mail(login=login, domain=domain)
                    code = self.code_mail(login, domain, id)
                    try:
                        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/input'))).send_keys(code)
                    except:
                        continue

                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/p'))).click()
                    break
                except:
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/form/div[7]/button'))).click()
                    continue

            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[2]/a'))).click()
            sleep(2)

            while True:
                for i in range(100):
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/span[1]/div/input'))).send_keys(self.mat_khau)
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/span[2]/div/input'))).send_keys(self.mat_khau)
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[2]/label/span/span'))).click()
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[3]/a'))).click()
                    sleep(2)
                    # Your existing code...
                    captcha_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div/div/div[2]/span/div/img')))
                    # Add this: wait until the element has non-zero size.
                    WebDriverWait(driver, 20).until(lambda d: captcha_element.size['width'] > 0 and captcha_element.size['height'] > 0)
                    # Continue with your existing code...
                    captcha_element.screenshot('captcha.png')
                    captcha_text = self.recognize_captcha('captcha.png')
                    captcha_numbers = ''.join(filter(str.isdigit, captcha_text))
                    captcha_input_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div/div/div[2]/span/div/input')))
                    captcha_input_element.clear()
                    sleep(1)
                    captcha_input_element.send_keys(captcha_numbers)
                    sleep(1)
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div/div/div[3]/a[1]'))).click()
                    sleep(1)
                    try:
                        captcha_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/section/p[1]')))
                        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//html/body/div[3]/section/p[3]/a'))).click()
                    except:
                        captcha_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/h3')))
                        break

                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[10]/label/span/span'))).click()
                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[11]/a[1]'))).click()

                driver.get('https://gfn.taiwanmobile.com/portal/payment/message?planProfileSid=20')
                driver.get('https://gfn.taiwanmobile.com/portal/payment/message?planProfileSid=20')

                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/section[1]/div[2]/div/div/a[1]'))).click()

                with open('acc.txt', 'a') as file:
                    file.write(str(self.mail) +  ' | ' + self.mat_khau.rstrip() + '\n')
                    print('| ' + self.mail + ' |' +' thành công !! ')

                driver.quit()

                print('NẾU MUỐN KẾT THÚC TOOLS THÌ:\n\nNhấn CLR + C để kết thúc chương trình\nNhấn CLR + C để kết thúc chương trình\n')

def tao_acc_process(x_position, y_position, width, height):
    x = GFN_TU_DONG(x_position, y_position, width, height)
    x.tao_acc()

if __name__ == '__main__':

    so_tien_trinh = int(input('Nhập số tiến trình: '))
    x_positions = [0, 300, 600]
    y_position = 0

    while True:

        running_processes = []

        try:
            while len(running_processes) < so_tien_trinh:
                x_position = x_positions[len(running_processes) % len(x_positions)]
                process = Process(target=tao_acc_process, args=(x_position, y_position, 300, 400))
                process.start()
                running_processes.append(process)
                time.sleep(0.5)


            for process in running_processes:
                process.join()

        except KeyboardInterrupt:
            print("| CHƯƠNG TRÌNH ĐÃ DỪNG !! |")
            break

In [ ]:
#@title Tool (Luồng / Threads)
import pytesseract
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import time
import os
import threading

def hide_errors():
    if os.name == 'posix':
        os.system('echo "\033[2J"')
    elif os.name == 'nt':
        os.system('cls')

hide_errors()

class GFN_TU_DONG:
    def __init__(self, x, y, width, height):
        self.mat_khau = 'hanamidz123'
        self.mail = ''
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.driver = None
        self.success_count = 0
        self.lock = threading.Lock()

    @staticmethod
    def recognize_captcha(captcha_image_path):
        captcha_image = Image.open(captcha_image_path)
        captcha_text = pytesseract.image_to_string(captcha_image)
        return captcha_text

    def get_mail(self):
        url = 'https://www.1secmail.com/api/v1/?action=genRandomMailbox&count=1'
        res = requests.get(url).json()
        self.mail = res[0]
        return self.mail

    def check_mail(self, login, domain):
        url = f'https://www.1secmail.com/api/v1/?action=getMessages&login={login}&domain={domain}'
        res = requests.get(url).json()
        id = res[0]['id']
        return id

    def code_mail(self, login, domain, id):
        url2 = f"https://www.1secmail.com/api/v1/?action=readMessage&login={login}&domain={domain}&id={id}"
        getip_new = requests.get(url2).json()
        body = getip_new['body']
        code_start = body.find('驗證碼')
        code = body[code_start:].split(':')[1].strip()
        return code

    def tao_acc(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        service = Service(executable_path=r'/usr/bin/chromedriver')
        driver = webdriver.Chrome(service=service,options=options)
        driver.set_window_rect(self.x, self.y, self.width, self.height)

        driver.get('https://gfn.taiwanmobile.com/portal/index')
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/nav/div[2]/ul[1]/div[2]/a[1]/li'))).click()
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/p[3]/a'))).click()
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/p[2]/a'))).click()
        self.get_mail()
        login, domain = self.mail.split('@')
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/span/div/input'))).send_keys(self.mail)
        print(self.mail)
        print('-----------------')

        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[2]/a'))).click()

        while True:
            for i in range(10):
                try:
                    sleep(10)
                    id = self.check_mail(login=login, domain=domain)
                    code = self.code_mail(login, domain, id)
                    try:
                        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/input'))).send_keys(code)
                    except:
                        continue

                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/p'))).click()
                    break
                except:
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/form/div[7]/button'))).click()
                    continue

            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[2]/a'))).click()
            sleep(2)

            while True:
                for i in range(100):
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/span[1]/div/input'))).send_keys(self.mat_khau)
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/span[2]/div/input'))).send_keys(self.mat_khau)
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[2]/label/span/span'))).click()
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[3]/a'))).click()
                    sleep(2)

                    captcha_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div/div/div[2]/span/div/img')))
                    captcha_element.screenshot('captcha.png')
                    captcha_text = self.recognize_captcha('captcha.png')
                    captcha_numbers = ''.join(filter(str.isdigit, captcha_text))
                    captcha_input_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div/div/div[2]/span/div/input')))
                    captcha_input_element.clear()
                    sleep(1)
                    captcha_input_element.send_keys(captcha_numbers)
                    sleep(1)
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div/div/div[3]/a[1]'))).click()
                    sleep(1)
                    try:
                        captcha_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/section/p[1]')))
                        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//html/body/div[3]/section/p[3]/a'))).click()
                    except:
                        captcha_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/h3')))
                        break

                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[10]/label/span/span'))).click()
                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/div/form/div/div[11]/a[1]'))).click()

                driver.get('https://gfn.taiwanmobile.com/portal/payment/message?planProfileSid=20')
                driver.get('https://gfn.taiwanmobile.com/portal/payment/message?planProfileSid=20')

                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/section[1]/div[2]/div/div/a[1]'))).click()

                with self.lock:
                    with open('acc.txt', 'a') as file:
                        file.write(str(self.mail) +  ' | ' + self.mat_khau.rstrip() + '\n')
                        print('| ' + self.mail + ' |' +' thành công !! ')

                driver.quit()

                print('NẾU MUỐN KẾT THÚC TOOLS THÌ:\n\nNhấn CLR + C để kết thúc chương trình\nNhấn CLR + C để kết thúc chương trình\n')

def tao_acc_thread(x_position, y_position, width, height):
    x = GFN_TU_DONG(x_position, y_position, width, height)
    x.tao_acc()

if __name__ == '__main__':
    so_luong = int(input('Nhập số luồng: '))
    x_positions = [0, 300, 600]
    y_position = 0

    while True:
        running_threads = []

        try:
            while len(running_threads) < so_luong:
                x_position = x_positions[len(running_threads) % len(x_positions)]
                thread = threading.Thread(target=tao_acc_thread, args=(x_position, y_position, 300, 400))
                thread.start()
                running_threads.append(thread)
                time.sleep(0.5)

            for thread in running_threads:
                thread.join()

        except KeyboardInterrupt:
            print("| CHƯƠNG TRÌNH ĐÃ DỪNG !! |")
            break


In [ ]:
#@title Download list Acc file

from google.colab import files
files.download('acc.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>